In [17]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

['test.csv', 'train.csv', 'gender_submission.csv']


In [18]:
train = pd.read_csv('../input/train.csv')
test = pd.read_csv('../input/test.csv')

#Save the PassengerId for later construction of the submission file
PassengerId = test['PassengerId']


train.head(3)

train.count() / len(train)

PassengerId    1.000000
Survived       1.000000
Pclass         1.000000
Name           1.000000
Sex            1.000000
Age            0.801347
SibSp          1.000000
Parch          1.000000
Ticket         1.000000
Fare           1.000000
Cabin          0.228956
Embarked       0.997755
dtype: float64

In [19]:
test.count() / len(test)

PassengerId    1.000000
Pclass         1.000000
Name           1.000000
Sex            1.000000
Age            0.794258
SibSp          1.000000
Parch          1.000000
Ticket         1.000000
Fare           0.997608
Cabin          0.217703
Embarked       1.000000
dtype: float64


Given that we have a description of the data columns and an understanding of the data quantity, we can make executive decisions about which columns to keep and about how to fill in gaps in the data:

    The PassengerId column can be removed from both train and test sets as it is not relevant to the analysis.
    In both sets, only about 22% of the data exists for the Cabin column, so we will drop that column from both.
    ~80% of the Age column exists for both sets, so we will impute the missing ages by replacing the NaN values with the gender mean.
    Values will be imputed for the missing Embarked data in Train and Fare data in test.

First, define a method--'drop_incomplete_cols'--which removes a column if less than a certain percentage of the data is present:


In [20]:
def drop_incomplete_cols(df):
    s = df.count() / len(df)
    threshold = .4
    for col_name in s.index:
        if s[col_name] < threshold:
            df.drop(col_name, axis=1, inplace=True)

In [21]:
all_data = [train, test]

for df in all_data:
    df.drop(['PassengerId'], axis=1, inplace=True)
    #impute missing Age with the mean across Sex
    df['Age'].fillna(df.groupby(['Sex'])['Age'].transform(np.mean), inplace=True)
    #impute missing Fare with the mean across Pclass and Embarked
    df['Fare'].fillna(df.groupby(['Pclass', 'Embarked'])['Fare'].transform(np.mean), inplace=True)
    #impute missing Embarked with mode of data set
    df['Embarked'].fillna(df['Embarked'].mode().iloc[0], inplace=True)
    drop_incomplete_cols(df)

In [22]:
train.count()/len(train)

Survived    1.0
Pclass      1.0
Name        1.0
Sex         1.0
Age         1.0
SibSp       1.0
Parch       1.0
Ticket      1.0
Fare        1.0
Embarked    1.0
dtype: float64

In [23]:
test.count()/len(test)

Pclass      1.0
Name        1.0
Sex         1.0
Age         1.0
SibSp       1.0
Parch       1.0
Ticket      1.0
Fare        1.0
Embarked    1.0
dtype: float64

In [24]:
(train.shape[0], test.shape[0])

(891, 418)